### This Kernel is an exploration of the various restaurants in major cities around the world. 

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import warnings
warnings.filterwarnings('ignore')

print(os.listdir("../input"))

In [ ]:
zom = pd.read_csv("../input/zomato.csv")

#### TOTAL RESTAURANTS AND ITS FEATURES

#### RESTAURANTS IN INDIA

In [ ]:
ind_res = zom[zom['Country Code'] == 1]
ind_res = ind_res.drop_duplicates(subset=None, keep='first', inplace=False)

### Which Restaurants in India has the highest & Lowest rating? 

In [ ]:
max_rate= ind_res.sort_values('Aggregate rating', ascending=False).groupby(by=['Country Code','City'],as_index=False).first() 
min_rate= ind_res.sort_values('Aggregate rating', ascending=False).groupby(by=['Country Code','City'],as_index=False).last() 
df_max= max_rate[['Country Code','City','Restaurant Name','Aggregate rating']]
df_min= min_rate[['Country Code','City','Restaurant Name','Aggregate rating']]

In [ ]:
rating_rest = df_max.merge(df_min,left_on='City',right_on='City',how='inner')
del rating_rest['Country Code_y']
rating_rest.columns=['Country Code','City','Highest Rated Restaurant','Rating Max','Lowest Rated Restaurant','Rating Min']

In [ ]:
city=rating_rest['City'].tolist()
rate_max=rating_rest['Rating Max'].tolist()
rate_min=rating_rest['Rating Min'].tolist()
rest_name_high=rating_rest['Highest Rated Restaurant'].tolist()
rest_name_low=rating_rest['Lowest Rated Restaurant'].tolist()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import * 
init_notebook_mode(connected=True)

trace0 = go.Scatter(
    x=rate_max,
    y=city,
    mode='markers',
    text=rest_name_high,
    name='High Rated Restaurants',
    marker=dict(
        color='rgba(156, 165, 196, 0.95)',
        line=dict(
            color='rgba(156, 165, 196, 1.0)',
            width=1,
        ),
        symbol='circle',
        size=16,
    )
)
trace1 = go.Scatter(
    x=rate_min,
    y=city,
    mode='markers',
    text=rest_name_low,
    name='Low Rated Restaurants',
    marker=dict(
        color='rgba(204, 204, 204, 0.95)',
        line=dict(
            color='rgba(217, 217, 217, 1.0)',
            width=1,
        ),
        symbol='circle',
        size=16,
    )
)

layout = go.Layout(
    xaxis=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=0.25,
        ticklen=8,
        tickwidth=4,
        tickcolor='#000'
    ),
    yaxis=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=0.25,
        ticklen=8,
        tickwidth=4,
        tickcolor='#000'
    )
)

data = [trace0, trace1]
layout = go.Layout(
    title="Restaurant Ratings - INDIA",
    autosize=False,
    width=800,
    height=1000,
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor='rgb(102, 102, 102)',
        titlefont=dict(
            color='rgb(204, 204, 204)'
        ),
        tickfont=dict(
            color='rgb(102, 102, 102)',
        ),
        automargin=True,
        showticklabels=True,
        dtick=1,
        ticklen=5,
        ticks='outside',
        tickcolor='rgb(102, 102, 102)',
    ),
    yaxis=dict(
        ticks='outside',
        tick0=0,
        dtick=1,
        ticklen=40,
        tickwidth=1,
        tickcolor='#000',
        automargin=True
    ),
    legend=dict(
        font=dict(
            size=10,
        ),
        yanchor='middle',
        xanchor='right',
    ),
    paper_bgcolor='rgb(254, 247, 234)',
    plot_bgcolor='rgb(254, 247, 234)',
    hovermode='closest',
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='rri')

#### DISTRIBUTION OF THE RATING VARIABLE

In [ ]:
sns.set_color_codes()

plt.figure(figsize=(16, 6))
ax = sns.distplot(ind_res['Aggregate rating'], color="y")

In [ ]:
rt_ind = ['Good', 'Very Good', 'Average', 'Excellent', 'Poor']
rt_ind_df = ind_res[ind_res['Rating text'].isin(rt_ind)]
#rt_ind_df

In [ ]:
def regioncolors(counter):
    if counter['Rating text'] in ['Excellent', 'Very Good']:
        return 'green'
    elif counter['Rating text'] == 'Good':
        return 'Yellow'
    elif counter['Rating text'] == 'Poor':
        return 'red'
    else:
        return 'blue'
rt_ind_df["Rating colors"] = rt_ind_df.apply(regioncolors, axis=1)
#rt_ind_df.head()

#### REGION-WISE EXPLORATION OF THE RESTARAUNTS

#### SOUTH INDIA

In [ ]:
#ind_res
south_ind = ['Chennai', 'Bangalore', 'Hyderabad', 'Vizag', 'Kochi', 'Mysore', 'Mangalore', 'Coimbatore', 'Ooty', 'Secunderabad']
south_ind_df = ind_res[ind_res['City'].isin(south_ind)]

south_ind_df["Rating colors"] = south_ind_df.apply(regioncolors, axis=1)

In [ ]:
locations = south_ind_df[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
#len(locationlist)

In [ ]:
import folium
from folium.plugins import MarkerCluster

map3 = folium.Map(location=[12.89, 77.59], tiles='CartoDB positron', zoom_start=11)

marker_cluster = MarkerCluster().add_to(map3)

for point in range(0, len(south_ind_df)):
    folium.Marker(locationlist[point], popup=south_ind_df['Locality'].iloc[point]+ ' '+ south_ind_df['Restaurant Name'].iloc[point] + ' ' + south_ind_df['City'].iloc[point], icon=folium.Icon(color=south_ind_df["Rating colors"].iloc[point], icon_color='white', icon='male', angle=0, prefix='fa'))\
    .add_to(marker_cluster)
map3


legend_html = '''
     <div style=”position: fixed; 
     bottom: 50px; left: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     “>&nbsp; <br>
     &nbsp; Excellent & Very Good &nbsp; <i class="fa fa-map-marker fa-2x"
                  style= "color:green"></i><br>
     &nbsp; Good &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:Yellow"></i><br>
     &nbsp; Average &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:blue"></i><br>
     &nbsp; Poor &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:red"></i><br><br>
                  
     &nbsp; Zoom in to explore each restaurant locality/street wise </i><br>
     &nbsp; Zoom out to take a look at the surrounding cities' data</i>
      </div>
     '''
map3.get_root().html.add_child(folium.Element(legend_html))
map3

#### NORTH INDIA

In [ ]:
#ind_res
north_ind = ['Agra', 'Amritsar', 'Dehradun', 'Chandigarh', 'Ghaziabad', 'Gurgaon', 'Indore', 'Lucknow', 'Ludhiana', 'Mohali', 'New  Delhi',\
             'Patna', 'Ranchi', 'Shimla', 'Varanasi', 'Zirakpur', 'Panchkula', 'Allahabad']
north_ind_df = ind_res[ind_res['City'].isin(north_ind)]

north_ind_df["Rating colors"] = north_ind_df.apply(regioncolors, axis=1)

In [ ]:
locations_n = north_ind_df[['Latitude', 'Longitude']]
locationlist_n = locations_n.values.tolist()
#len(locationlist_n)

In [ ]:
import folium
from folium.plugins import MarkerCluster

map3 = folium.Map(location=[27.16, 78.01], tiles='cartodbdark_matter', zoom_start=11)

marker_cluster = MarkerCluster().add_to(map3)

for point in range(0, len(north_ind_df)):
    folium.Marker(locationlist_n[point], popup=north_ind_df['Locality'].iloc[point]+ ' '+ north_ind_df['Restaurant Name'].iloc[point] + ' ' + north_ind_df['City'].iloc[point], icon=folium.Icon(color=north_ind_df["Rating colors"].iloc[point], icon_color='white', icon='male', angle=0, prefix='fa'))\
    .add_to(marker_cluster)
map3

legend_html = '''
     <div style=”position: fixed; 
     bottom: 50px; left: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     “>&nbsp; <br>
     &nbsp; Excellent & Very Good &nbsp; <i class="fa fa-map-marker fa-2x"
                  style= "color:green"></i><br>
     &nbsp; Good &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:Yellow"></i><br>
     &nbsp; Average &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:blue"></i><br>
     &nbsp; Poor &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:red"></i><br><br>
                  
     &nbsp; Zoom in to explore each restaurant locality/street wise </i><br>
     &nbsp; Zoom out to take a look at the surrounding cities' data</i>
      </div>
     '''
map3.get_root().html.add_child(folium.Element(legend_html))
map3

#### Western India

In [ ]:
#ind_res
west_ind = ['Ahmedabad', 'Bhubaneshwar', 'Goa', 'Jaipur', 'Nagpur', 'Nashik', 'Nasik', 'Pune', 'Surat', 'Thane', 'Mumbai',\
             'Vadodara']
west_ind_df = ind_res[ind_res['City'].isin(west_ind)]

west_ind_df["Rating colors"] = west_ind_df.apply(regioncolors, axis=1)

In [ ]:
locations_w = west_ind_df[['Latitude', 'Longitude']]
locationlist_w = locations_w.values.tolist()
#len(locationlist_w)

In [ ]:
import folium
from folium.plugins import MarkerCluster

map3 = folium.Map(location=[23.03, 72.53], tiles='stamenterrain', zoom_start=11)

marker_cluster = MarkerCluster().add_to(map3)

for point in range(0, len(west_ind_df)):
    folium.Marker(locationlist_w[point], popup=west_ind_df['Locality'].iloc[point]+ ' '+ west_ind_df['Restaurant Name'].iloc[point] + ' ' + west_ind_df['City'].iloc[point], icon=folium.Icon(color=west_ind_df["Rating colors"].iloc[point], icon_color='white', icon='male', angle=0, prefix='fa'))\
    .add_to(marker_cluster)
map3

legend_html = '''
     <div style=”position: fixed; 
     bottom: 50px; left: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     “>&nbsp; <br>
     &nbsp; Excellent & Very Good &nbsp; <i class="fa fa-map-marker fa-2x"
                  style= "color:green"></i><br>
     &nbsp; Good &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:Yellow"></i><br>
     &nbsp; Average &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:blue"></i><br>
     &nbsp; Poor &nbsp; <i class="fa fa-map-marker fa-2x"
                  style="color:red"></i><br><br>
                  
     &nbsp; Zoom in to explore each restaurant locality/street wise </i><br>
     &nbsp; Zoom out to take a look at the surrounding cities' data</i>
      </div>
     '''
map3.get_root().html.add_child(folium.Element(legend_html))
map3

 ## CUISINES

In [ ]:
#zom['Cuisines'].unique()
zomato_cui= zom[zom['Cuisines'].notnull()]
zomato_cui.shape

In [ ]:
cuisines = zomato_cui['Cuisines'].apply(lambda x: pd.Series(x.split(',')))
cuisines.columns=['Cuisine 1','Cuisine 2','Cuisine 3','Cuisine 4','Cuisine 5','Cuisine 6','Cuisines 7','Cuisine 8']# renaming the columns

In [ ]:
result = pd.concat([zom, cuisines], axis=1) #concating the two dataframes 

In [ ]:
locality=pd.DataFrame(result[['Locality Verbose','Cuisine 1','Cuisine 2','Cuisine 3','Cuisine 4','Cuisine 5','Cuisine 6','Cuisines 7','Cuisine 8']])
locality_stack=pd.DataFrame(locality.stack()) #stacking the columns 
#locality.head()

In [ ]:
keys = [c for c in locality  if c.startswith('Cuisine')]
a=pd.melt(locality, id_vars='Locality Verbose', value_vars=keys, value_name='Cuisines')
max_rate=pd.DataFrame(a.groupby(by=['Locality Verbose','variable','Cuisines']).size().reset_index())

del max_rate['variable']
max_rate.columns=['Locality Verbose','Cuisines','Count']
#max_rate.head()

In [ ]:
loc=max_rate.sort_values('Count', ascending=False).groupby(by=['Locality Verbose'],as_index=False).first() #find the highest restuarant in the city

In [ ]:
rating_res = loc.merge(zom,left_on='Locality Verbose',right_on='Locality Verbose',how='inner')
df = pd.DataFrame(rating_res[['Country Code','Locality Verbose','Cuisines_x','Count']]) #making a dataframe of rating restaurant
country = rating_res.sort_values('Count', ascending=False).groupby(by=['Country Code'],as_index=False).first()
con = pd.DataFrame(country[['Country Code','Locality Verbose','Cuisines_x','Count']])
con.columns = ['Country Code','Locality','Cuisines','Number of restaurants in the country']#renaming the columns
con1 = con.sort_values('Number of restaurants in the country', ascending=False)
con1[:10]
final_con=con1.drop(con1.index[[7,10]])

#### LOCALITY WITH HIGHEST NUMBER OF RESTAURANTS ALONG WITH CUSININES

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=final_con['Locality'],
    y=final_con['Number of restaurants in the country'],
    text=final_con['Cuisines'],
    #width=500,
    #height=500,
)

data = [trace1]
layout = go.Layout(
    barmode='group',
    height=800
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')

### TOP CUISINES IN THE WORLD

In [ ]:
import seaborn as sns

my_range=range(1,len(final_con.index)+1)

plt.figure(figsize=(16, 6))

plt.hlines(y=my_range, xmin=0, xmax=final_con['Number of restaurants in the country'], color='skyblue')
plt.plot(final_con['Number of restaurants in the country'], my_range, "o")
 
plt.yticks(my_range, final_con['Cuisines'])
plt.title("Top Cuisines of the World", loc='left')
plt.xlabel('Number of restaurants in the country')
plt.ylabel('Cuisines')

#### TOP CUISINES IN INDIA

In [ ]:
rating_res=loc.merge(zom,left_on='Locality Verbose',right_on='Locality Verbose',how='inner')
rating_res1=rating_res[rating_res['Country Code']==1]
rating_restaurant=pd.DataFrame(rating_res1[['Locality Verbose','Cuisines_x','Count']])
rating_restaurant=rating_restaurant.sort_values('Count', ascending=False).groupby(by=['Cuisines_x'],sort=False,as_index=False).first()#grouping it for cuisines
rating_restaurant.columns=['Cuisines','Locality Verbose','Number of restaurants in the locality']
rating_restaurant1=rating_restaurant[rating_restaurant['Number of restaurants in the locality']>3]
rating_restaurant1
rating_restaurant_t6 = rating_restaurant1[:10]

In [ ]:
plt.figure(figsize=(16, 8))

sns.pointplot(x="Cuisines", y="Number of restaurants in the locality", data=rating_restaurant_t6)

### Average Cost for 2 - Region Wise

In [ ]:
west_ind.append('Aurangabad')
south_ind.append('Puducherry')
north_ind.append('Noida')
west_ind.append('Kanpur')

In [ ]:
def region(counter):
    if counter['City'] in west_ind:
        return 'West'
    elif counter['City'] in north_ind:
        return 'North'
    elif counter['City'] in south_ind:
        return 'South'
    else:
        return 'East'

In [ ]:
ind_res["Region"] = ind_res.apply(region, axis=1)

reg_avg = ind_res.groupby('Region') \
       .agg({'Restaurant Name':'size', 'Average Cost for two':'mean'}) \
       .rename(columns={'Restaurant Name':'Total Restaurants','Average Cost for two':'Average cost for 2'}) \
       .reset_index()

In [ ]:
plt.figure(figsize=(16, 8))

plt.ylim(0, 1500)
sns.boxenplot(x="Region", y="Average Cost for two", data=ind_res)

### Distribution of the cost variable

In [ ]:
plt.figure(figsize=(16, 8))

sns.distplot(ind_res['Average Cost for two'])

### Are ratings & cost related? 

#### DENSITY PLOT

In [ ]:
with sns.axes_style('white'):
    sns.jointplot(x="Aggregate rating", y="Average Cost for two", data=ind_res, kind="kde", height = 10)

#### POINT PLOT

In [ ]:
with sns.axes_style('white'):
    sns.jointplot(x="Aggregate rating", y="Average Cost for two", data=ind_res, height = 8)

#### POINT PLOT TELLS ME THERE IS 1 RESTAURANT THAT COSTS ~6000 for 2 people. Woah! 

#### How does it look in the Southern region?

In [ ]:
south_res = ind_res[ind_res['City'].isin(south_ind)]

with sns.axes_style('white'):
    sns.jointplot(x="Aggregate rating", y="Average Cost for two", data=south_res, height = 8)

#### How does it look in the Northern region?

In [ ]:
north_res = ind_res[ind_res['City'].isin(north_ind)]

with sns.axes_style('white'):
    sns.jointplot(x="Aggregate rating", y="Average Cost for two", data=north_res, height = 9)

#### Western India?

In [ ]:
west_res = ind_res[ind_res['City'].isin(west_ind)]

with sns.axes_style('white'):
    sns.jointplot(x="Aggregate rating", y="Average Cost for two", data=west_res, height = 9)

### Average Number of Votes - Region wise

In [ ]:
reg_votes = ind_res.groupby('Region') \
       .agg({'Restaurant Name':'size', 'Votes':'mean'}) \
       .rename(columns={'Restaurant Name':'Total Restaurants','Votes':'Votes'}) \
       .reset_index()

reg_votes['Votes'] = round(reg_votes['Votes'])

In [ ]:
plt.figure(figsize=(10, 8))
plt.ylim(0, 5000)
sns.violinplot(x="Region", y="Votes", data=ind_res)

#### THE PLOT IS DENSER FOR SOUTH THAN THE REST, EVEN ABOVE 3000 Range. 

### PAISA VASOOL Restaurant - Region Wise

umm, Paisa vasool you ask? -- https://www.quora.com/What-is-the-meaning-of-Hindi-word-Paisa-Vasool

### CRITERIA -- 400 or more Votes; 4 or more Rating ; Cost < 1000 for 2

### CHENNAI

In [ ]:
chennai = south_res[south_res['City'] == 'Chennai']

fc = chennai[(chennai['Votes'] >= 400) & (chennai['Aggregate rating'] >= 4) & (chennai['Average Cost for two'] <= 1000) ]

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=fc['Latitude'],
        lon=fc['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14
        ),
        text=fc['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=12.99,
            lon=80.2
        ),
        pitch=0,
        zoom=10.5,
        style='outdoors'
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='Chennai Mapbox')

### BANGALORE

In [ ]:
Bangalore = south_res[south_res['City'] == 'Bangalore']

fb = Bangalore[(Bangalore['Votes'] >= 400) & (Bangalore['Aggregate rating'] >= 4) & (Bangalore['Average Cost for two'] <= 1000) ]

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)
mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=fb['Latitude'],
        lon=fb['Longitude'],
        customdata = fb['Restaurant Name'],
        mode='markers',
        marker=dict(
                size=14,
                color = 'violet',
                opacity = .8),
        #marker=go.scattermapbox.Marker(
         #   size=14
        #),
        text=fb['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=False,
    #hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        pitch=10,
        zoom=10.5,
        center=dict(lat=12.96,
            lon=77.73),
        style= "mapbox://styles/akashram/cjumtx5010gjr1fmy13uzs47c" # "mapbox://styles/akashram/cjumqsvzu1xph1gqwzu42fnpt"
    ),
    width=900,
    height=600, 
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='Bangalore Mapbox')

### Hyderabad

In [ ]:
hb = south_res[south_res['City'] == 'Hyderabad']

fhb = hb[(hb['Votes'] >= 400) & (hb['Aggregate rating'] >= 4) & (hb['Average Cost for two'] <= 1000) ]

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=fhb['Latitude'],
        lon=fhb['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color = 'yellow'
        ),
        text=fhb['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=17.4,
            lon=78.4
        ),
        pitch=0,
        zoom=10.5,
        style= "mapbox://styles/akashram/cjumu0d6n0gmv1fpk24lqh60s" #'streets'
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='Hyd Mapbox')

### New Delhi

In [ ]:
nd = ind_res[ind_res['City'] == 'New Delhi']

fnd = nd[(nd['Votes'] >= 400) & (nd['Aggregate rating'] >= 4) & (nd['Average Cost for two'] <= 1000) ]

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=fnd['Latitude'],
        lon=fnd['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color = 'Purple',
            opacity = .8
        ),
        text=fnd['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=28.6,
            lon=77.3
        ),
        pitch=0,
        zoom=10.5,
        style = "mapbox://styles/akashram/cjumudmfh0gyw1fo4yb0mkisl"
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='ND Mapbox')

### MUMBAI

In [ ]:
mb = ind_res[ind_res['City'] == 'Mumbai']

fm = mb[(mb['Votes'] >= 400) & (mb['Aggregate rating'] >= 4) & (mb['Average Cost for two'] <= 1000) ]

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=fm['Latitude'],
        lon=fm['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color='white'
        ),
        text=fm['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=19.1,
            lon=72.8
        ),
        pitch=0,
        zoom=10.5,
        style='mapbox://styles/akashram/cjumujrxr0h0r1fodd4o3xh60'
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='Mum Mapbox')

### Goa

In [ ]:
goa = ind_res[ind_res['City'] == 'Goa']

fg = goa[(goa['Votes'] >= 400) & (goa['Aggregate rating'] >= 4) & (goa['Average Cost for two'] <= 1000) ]

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'
#MapboxAccess.ClearCache()
data = [
    go.Scattermapbox(
        lat=fg['Latitude'],
        lon=fg['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color='red'
        ),
        text=fg['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=15.5,
            lon=73.7
        ),
        pitch=0,
        zoom=10.5,
        style= 'mapbox://styles/akashram/cjumup1vt0h9t1fqp5lk9otm9'#'mapbox://styles/akashram/cjumumolo0fit1glh02jdehm6' #'outdoors'
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='Goa Mapbox')

### OTHER CITIES

In [ ]:
london = zom[zom['City'] == 'London']

#### How does the aggregate rating look like for top restaurants in London?

In [ ]:
sns.set_color_codes()

plt.figure(figsize=(16, 6))
ax = sns.distplot(london['Aggregate rating'], color="g")

#### How does rating relate to cost for 2 (Pounds) ?

In [ ]:
with sns.axes_style('white'):
    sns.jointplot(x="Aggregate rating", y="Average Cost for two", data=london, height = 9)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=london['Latitude'],
        lon=london['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color='Pink'
        ),
        text=london['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lon=0.02,
            lat=51.5
        ),
        pitch=0,
        zoom=10.5,
        style= 'mapbox://styles/akashram/cjumz93b108ke1fqbc2vgnxkv' #'outdoors'
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='london Mapbox')

### LAS VEGAS

In [ ]:
lv = zom[zom['City'] == 'Las Vegas']

In [ ]:
sns.set_color_codes()

plt.figure(figsize=(16, 6))
ax = sns.distplot(lv['Aggregate rating'], color="b")

In [ ]:
with sns.axes_style('white'):
    sns.jointplot(x="Aggregate rating", y="Average Cost for two", data=lv, height = 9)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=lv['Latitude'],
        lon=lv['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color='Yellow'
        ),
        text=lv['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lon=-115.2,
            lat=36.12
        ),
        pitch=0,
        zoom=11.5,
        style= 'dark'
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='london Mapbox')

### SINGAPORE

In [ ]:
sg = zom[zom['City'] == 'Singapore']

In [ ]:
sns.set_color_codes()

plt.figure(figsize=(16, 6))
ax = sns.distplot(sg['Aggregate rating'], color="r")

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

mapbox_access_token = 'pk.eyJ1IjoiYWthc2hyYW0iLCJhIjoiY2p1bW9sMGt2MGF2NzQ1cG44aGJtbm1qZCJ9.4TgvYSRAD8md9DM_nPOFWQ'

data = [
    go.Scattermapbox(
        lat=sg['Latitude'],
        lon=sg['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=14,
            color='Violet'
        ),
        text=sg['Restaurant Name'],
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lon=103.8,
            lat=1.3
        ),
        pitch=0,
        zoom=12,
        style= 'streets'
    ),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='london Mapbox')

#### Please Upvote if you liked the kernel. Lot more can be done & I think you can explore more, with the updated dataset every week.